In [1]:
from tkinter import messagebox
import numpy as np
import pandas as pand
import os
import cv2
import random
import base64

from src.helper.file import File
from src.helper.cipher import encrypt_vigenere

In [2]:
pip install opencv-python


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
Wc = np.indices((8, 8)).sum(axis=0) % 2


class Inserter:
#     def __init__(self, file_dir, secret_message_dir, key):
#         image_file = File(file_dir)
#         self.ndarray = image_file.read_ndarray_image_file()
#         self.h, self.w, self.color = self.ndarray.shape

#         secret_message = File(secret_message_dir)
#         self.extension = secret_message.get_extention()
#         self.string_message = ""

#         byte_message = secret_message.read_files()
#         self.message = base64.b64encode(byte_message).decode('utf-8')

#         self.key = key

    def __init__(self, image, message, key):
        self.ndarray = image
        self.h, self.w, self.color = self.ndarray.shape

        self.extension = '.txt'
        self.string_message = ""

        byte_message = message
        self.message = base64.b64encode(byte_message).decode('utf-8')

        self.key = key

    def count_seed(self):
        return sum([ord(i) for i in self.key])

    def encrypt_message(self, encrypted, key):
        sign = 1 if encrypted else 0

        self.ndarray[0][0][0] = self.ndarray[0][0][0] & 254 | sign
        if encrypted:
            self.string_message = encrypt_vigenere(self.string_message, key)

    def random_list(self, randomize_frames):
        sign = 1 if randomize_frames else 0

        self.ndarray[0][0][1] = self.ndarray[0][0][1] & 254 | sign
        if randomize_frames:
            random.seed(self.seed)
            if self.method == 'lsb':
                random.shuffle(self.pixel_list)
            elif self.method == 'bpcs':
                random.shuffle(self.block_list)

    def modify_pixel(self, array_bit):
        index = 0
        for i in self.pixel_list:
            if index >= len(array_bit):
                break
            if i >= 3:
                h, w, color = self.get_ndarray_pos(i)
                self.ndarray[h][w][color] = self.ndarray[h][w][color] & 254 | array_bit[index]
                index += 1
        if index < len(array_bit):
            error = "Ukuran pesan melebihi kapasitas payload!"
            messagebox.showerror("Kesalahan", error)
            raise RuntimeError(error)

    def insert_alpha(self):
        alpha_str = str(self.alpha)[:7].ljust(7, '0')
        alpha_bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8, '0') for i in alpha_str])))
        index = 0
        for i in range(1, 8):
            for j in range(8):
                self.ndarray[i][j][0] = self.ndarray[i][j][0] & 254 | alpha_bits[index]
                index += 1

    def pbc_to_cgc(self):
        b = self.ndarray
        g = b >> 7
        for i in range(7, 0, -1):
            g <<= 1
            g |= ((b >> i) & 1) ^ ((b >> (i - 1)) & 1)
        self.ndarray = g

    def cgc_to_pbc(self):
        g = self.ndarray
        b = g >> 7
        for i in range(7, 0, -1):
            b_before = b.copy()
            b <<= 1
            b |= (b_before & 1) ^ ((g >> (i - 1)) & 1)
        self.ndarray = b

    def complexity(self, matrix):
        maxim = ((matrix.shape[0] - 1) * matrix.shape[1]) + ((matrix.shape[1] - 1) * matrix.shape[0])
        curr = 0.0
        first = matrix[0,0]
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if (first != matrix[i,j]):
                    curr = curr + 1
                    first = matrix[i,j]
        first = matrix[0,0]
        for i in range(matrix.shape[1]):
            for j in range(matrix.shape[0]):
                if (first != matrix[j,i]):
                    curr = curr + 1
                    first = matrix[j,i]
        return curr/maxim
    
    def conjugate(self, P):
        return P ^ Wc

    def modify_block(self, array_bit):
        index = 0
        for bitplane in range(7, -1, -1):
            # bitplane 0 = msb, 7 = lsb
            for h, w, color in self.block_list:
                if index >= len(array_bit):
                    break
                if h != 0 and w != 0 and color != 0:
                    matrix = self.ndarray[h:h+8, w:w+8, color] >> (7 - bitplane) & 1
                    if self.complexity(matrix) > self.alpha:
                        current_bit = np.zeros((8, 8), dtype=int)
                        for i in range(8):
                            for j in range(8):
                                if (i > 0 or j > 0) and index < len(array_bit):
                                    current_bit[i, j] = array_bit[index]
                                    index += 1

                        if self.complexity(current_bit) <= self.alpha:
                            current_bit = self.conjugate(current_bit)
                            current_bit[0, 0] = 1

                        left = (self.ndarray[h:h+8, w:w+8, color] >> (8 - bitplane)) << (8 - bitplane)
                        right = self.ndarray[h:h+8, w:w+8, color] & ((1 << (8 - bitplane - 1)) - 1)
                        self.ndarray[h:h+8, w:w+8, color] = left + (current_bit << 7 - bitplane) + right
            if index >= len(array_bit):
                break
        if index < len(array_bit):
            error = "Ukuran pesan melebihi kapasitas payload!"
            messagebox.showerror("Kesalahan", error)
            raise RuntimeError(error)

    def get_ndarray_pos(self, idx):
        color = idx % self.color
        w = (idx // self.color) % self.w
        h = idx // (self.color * self.w)
        return h, w, color

    def insert_message(self, encrypted=False, randomize=False, method='bpcs', alpha=0.3):
        self.seed = self.count_seed()
        self.method = method
        self.alpha = alpha

        self.string_message = str(len(self.message)) + '#' + self.extension + '#' + self.message
        self.encrypt_message(encrypted, self.key)

        bits = map(int, ''.join(
            [bin(ord(i)).lstrip('0b').rjust(8, '0') for i in self.string_message]))
        array_bit = list(bits)

        #for i in self.frame:
        if method == 'lsb':
            self.pixel_list = list(range(self.h * self.w * self.color))
            self.random_list(randomize)
            self.modify_pixel(array_bit)
            self.ndarray[0, 0, 2] = self.ndarray[0, 0, 2] & 254
        elif method == 'bpcs':
            self.block_list = []
            for h in range(0, self.h - (self.h % 8), 8):
                for w in range(0, self.w - (self.w % 8), 8):
                    for color in range(0, self.color):
                        self.block_list += [(h, w, color)]
            self.random_list(randomize)
            self.pbc_to_cgc()
            self.modify_block(array_bit)
            self.cgc_to_pbc()
            self.insert_alpha()
            self.ndarray[0, 0, 2] = self.ndarray[0, 0, 2] & 254 | 1

        return self.ndarray

In [4]:
file = './sample/image/blue_sea.png'
message = './sample/text/short.txt'
key = 'anu'

insert = Inserter(file, message, key)

inserted_image = insert.insert_message(encrypted=True, randomize=False, method='bpcs', alpha=0.3)

output = './blue_seajadi.png'
output_file = File(output)
output_file.write_image_file(inserted_image)

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
video = load_video('./sample/video/ex2.avi')

frames = get_frames_from_video(video)

frames_len = len(frames)
frames_len

In [19]:
fourcc = get_fourcc(video)
print(fourcc)

FMP4


In [20]:
message = load_text('./sample/text/message.txt')
message_len = len(message)
message_len

2945

In [21]:
char_per_frame = 1
if message_len > frames_len:
    char_per_frame = int(math.ceil(message_len / frames_len))
    
char_per_frame

5

In [22]:
key = 'anu'

inserted_frames = []

current_message_index = 0
for i in range(frames_len):
    current_message = message[current_message_index:(current_message_index+char_per_frame)]
    current_message_bytes = bytes(current_message, 'utf-8')
    insert = Inserter(frames[i], current_message_bytes, key)

    inserted_frame = insert.insert_message(encrypted=True, randomize=False, method='bpcs', alpha=0.3)
    inserted_frames.append(inserted_frame)

    current_message_index += char_per_frame
    print('Menyisipkan pesan di frame ke-', i+1)

Menyisipkan pesan di frame ke- 1
Menyisipkan pesan di frame ke- 2
Menyisipkan pesan di frame ke- 3
Menyisipkan pesan di frame ke- 4
Menyisipkan pesan di frame ke- 5
Menyisipkan pesan di frame ke- 6
Menyisipkan pesan di frame ke- 7
Menyisipkan pesan di frame ke- 8
Menyisipkan pesan di frame ke- 9
Menyisipkan pesan di frame ke- 10
Menyisipkan pesan di frame ke- 11
Menyisipkan pesan di frame ke- 12
Menyisipkan pesan di frame ke- 13
Menyisipkan pesan di frame ke- 14
Menyisipkan pesan di frame ke- 15
Menyisipkan pesan di frame ke- 16
Menyisipkan pesan di frame ke- 17
Menyisipkan pesan di frame ke- 18
Menyisipkan pesan di frame ke- 19
Menyisipkan pesan di frame ke- 20
Menyisipkan pesan di frame ke- 21
Menyisipkan pesan di frame ke- 22
Menyisipkan pesan di frame ke- 23
Menyisipkan pesan di frame ke- 24
Menyisipkan pesan di frame ke- 25
Menyisipkan pesan di frame ke- 26
Menyisipkan pesan di frame ke- 27
Menyisipkan pesan di frame ke- 28
Menyisipkan pesan di frame ke- 29
Menyisipkan pesan di fr

Menyisipkan pesan di frame ke- 239
Menyisipkan pesan di frame ke- 240
Menyisipkan pesan di frame ke- 241
Menyisipkan pesan di frame ke- 242
Menyisipkan pesan di frame ke- 243
Menyisipkan pesan di frame ke- 244
Menyisipkan pesan di frame ke- 245
Menyisipkan pesan di frame ke- 246
Menyisipkan pesan di frame ke- 247
Menyisipkan pesan di frame ke- 248
Menyisipkan pesan di frame ke- 249
Menyisipkan pesan di frame ke- 250
Menyisipkan pesan di frame ke- 251
Menyisipkan pesan di frame ke- 252
Menyisipkan pesan di frame ke- 253
Menyisipkan pesan di frame ke- 254
Menyisipkan pesan di frame ke- 255
Menyisipkan pesan di frame ke- 256
Menyisipkan pesan di frame ke- 257
Menyisipkan pesan di frame ke- 258
Menyisipkan pesan di frame ke- 259
Menyisipkan pesan di frame ke- 260
Menyisipkan pesan di frame ke- 261
Menyisipkan pesan di frame ke- 262
Menyisipkan pesan di frame ke- 263
Menyisipkan pesan di frame ke- 264
Menyisipkan pesan di frame ke- 265
Menyisipkan pesan di frame ke- 266
Menyisipkan pesan di

KeyboardInterrupt: 

In [ ]:
fps = get_video_fps(video)
output_filepath = './inserted_ex2_correctfourccfpscolor.avi'

save_frames_to_video(inserted_frames, fps, fourcc, output_filepath)

In [23]:
for i in range(frames_len):
    filename = './tmp/frame' + str(i) + '.png'
    cv2.imwrite(filename, inserted_frames[i])
    print('Berhasil menyimpan frame ke-', i+1)

Berhasil menyimpan frame ke- 1
Berhasil menyimpan frame ke- 2
Berhasil menyimpan frame ke- 3
Berhasil menyimpan frame ke- 4
Berhasil menyimpan frame ke- 5
Berhasil menyimpan frame ke- 6
Berhasil menyimpan frame ke- 7
Berhasil menyimpan frame ke- 8
Berhasil menyimpan frame ke- 9
Berhasil menyimpan frame ke- 10
Berhasil menyimpan frame ke- 11
Berhasil menyimpan frame ke- 12
Berhasil menyimpan frame ke- 13
Berhasil menyimpan frame ke- 14
Berhasil menyimpan frame ke- 15
Berhasil menyimpan frame ke- 16
Berhasil menyimpan frame ke- 17
Berhasil menyimpan frame ke- 18
Berhasil menyimpan frame ke- 19
Berhasil menyimpan frame ke- 20
Berhasil menyimpan frame ke- 21
Berhasil menyimpan frame ke- 22
Berhasil menyimpan frame ke- 23
Berhasil menyimpan frame ke- 24
Berhasil menyimpan frame ke- 25
Berhasil menyimpan frame ke- 26
Berhasil menyimpan frame ke- 27
Berhasil menyimpan frame ke- 28
Berhasil menyimpan frame ke- 29
Berhasil menyimpan frame ke- 30
Berhasil menyimpan frame ke- 31
Berhasil menyimpa

IndexError: list index out of range

In [ ]:
from subprocess import call,STDOUT

call(["ffmpeg", "-i", "tmp/frame%d.png" , "-vcodec", "png", "inserted_ex2_new.avi", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)

In [39]:
# from src.helper.cipher import decrypt_vigenere

In [40]:
# Wc = np.indices((8, 8)).sum(axis=0) % 2


# class Extractor:
# #     def __init__(self, file_dir, key):
# #         stegano_image_file = File(file_dir)
# #         self.ndarray = stegano_image_file.read_ndarray_image_file()
# #         self.h, self.w, self.color = self.ndarray.shape
# #         self.key = key

#     def __init__(self, image, key):
#         self.ndarray = image
#         self.h, self.w, self.color = self.ndarray.shape
#         self.key = key

#     def conjugate(self, P):
#         return P ^ Wc

#     def pbc_to_cgc(self):
#         b = self.ndarray
#         g = b >> 7
#         for i in range(7, 0, -1):
#             g <<= 1
#             g |= ((b >> i) & 1) ^ ((b >> (i - 1)) & 1)
#         self.ndarray = g

#     def cgc_to_pbc(self):
#         g = self.ndarray
#         b = g >> 7
#         for i in range(7, 0, -1):
#             b_before = b.copy()
#             b <<= 1
#             b |= (b_before & 1) ^ ((g >> (i - 1)) & 1)
#         self.ndarray = b

#     def complexity(self, matrix):
#         maxim = ((matrix.shape[0] - 1) * matrix.shape[1]) + ((matrix.shape[1] - 1) * matrix.shape[0])
#         curr = 0.0
#         first = matrix[0,0]
#         for i in range(matrix.shape[0]):
#             for j in range(matrix.shape[1]):
#                 if (first != matrix[i,j]):
#                     curr = curr + 1
#                     first = matrix[i,j]
#         first = matrix[0,0]
#         for i in range(matrix.shape[1]):
#             for j in range(matrix.shape[0]):
#                 if (first != matrix[j,i]):
#                     curr = curr + 1
#                     first = matrix[j,i]
#         return curr/maxim

#     def count_seed(self):
#         return sum([ord(i) for i in self.key])

#     def get_ndarray_pos(self, idx):
#         color = idx % self.color
#         w = (idx // self.color) % self.w
#         h = idx // (self.color * self.w)
#         return h, w, color

#     def extract_alpha(self):
#         index = 0
#         mod_index = 8
#         temp = ""
#         alpha_str = ""

#         for i in range(1, 8):
#             for j in range(8):
#                 extracted = self.ndarray[i][j][0] & 1
#                 if index % mod_index != (mod_index - 1):
#                     temp += str(extracted)
#                 else:
#                     temp += str(extracted)
#                     alpha_str += chr(int(temp, 2))
#                     temp = ""
#                 index += 1
#         return float(alpha_str)

#     def extract_messages(self):
#         self.seed = self.count_seed()

#         extracted = [self.ndarray[self.get_ndarray_pos(i)] & 1 for i in range(self.h * self.w * self.color)]
#         encrypted = extracted[0]
#         random_pixels = extracted[1]
#         method = 'bpcs' if extracted[2] else 'lsb'

#         index = 0
#         mod_index = 8

#         message = ""
#         temp = ""

#         if method == 'lsb':
#             pixel_list = list(range(len(extracted)))
#             if random_pixels:
#                 random.seed(self.seed)
#                 random.shuffle(pixel_list)

#             for i in pixel_list:
#                 if i >= 3:
#                     if index % mod_index != (mod_index - 1):
#                         temp += str(extracted[i])
#                     else:
#                         temp += str(extracted[i])
#                         message += chr(int(temp, 2))
#                         temp = ""
#                     index += 1
#         elif method == 'bpcs':
#             alpha = self.extract_alpha()
#             block_list = []
#             for h in range(0, self.h - (self.h % 8), 8):
#                 for w in range(0, self.w - (self.w % 8), 8):
#                     for color in range(0, self.color):
#                         block_list += [(h, w, color)]
#             if random_pixels:
#                 random.seed(self.seed)
#                 random.shuffle(block_list)
#             self.pbc_to_cgc()
#             for bitplane in range(7, -1, -1):
#                 for h, w, color in block_list:
#                     if h != 0 and w != 0 and color != 0:
#                         matrix = self.ndarray[h:h+8, w:w+8, color] >> (7 - bitplane) & 1
#                         if self.complexity(matrix) > alpha:
#                             if matrix[0, 0]:
#                                 matrix = self.conjugate(matrix)
#                             for i in range(8):
#                                 for j in range(8):
#                                     if i > 0 or j > 0:
#                                         extracted_bit = matrix[i][j]
#                                         if index % mod_index != (mod_index - 1):
#                                             temp += str(extracted_bit)
#                                         else:
#                                             temp += str(extracted_bit)
#                                             message += chr(int(temp, 2))
#                                             temp = ""
#                                         index += 1

#         if encrypted:
#             self.string_message = decrypt_vigenere(message, self.key)
#         else:
#             self.string_message = message

#     def parse_message(self):
#         message_info = self.string_message.split("#")

#         try:
#             self.len_message = int(message_info[0])
#             self.extension = message_info[1]
#         except:
#             self.len_message = 0
#             self.extension = 'txt'
        

#     def write_secret_message(self):
#         init = len(str(self.len_message)) + len(str(self.extension)) + 2
#         decoded = self.string_message[init : init + self.len_message]

#         bytes_file = decoded.encode('utf-8')
#         bytes_file = base64.b64decode(bytes_file)

#         return bytes_file


In [41]:
#extract = Extractor(inserted_frames[0], key)
#extract.extract_messages()
#extract.parse_message()

#byte = extract.write_secret_message()
#current_message = byte.decode('utf-8')
#print(current_message)

In [42]:
#key = 'anu'
#message = ''

#for i in range(frames_len):
#    extract = Extractor(inserted_frames[i], key)
#    extract.extract_messages()
#    extract.parse_message()

#    byte = extract.write_secret_message()
#    current_message = byte.decode('utf-8')
#    message += current_message
#    print('Ekstraksi pesan di frame ke-', i+1, 'yaitu=', current_message)

    
#print('Pesan rahasia:', message)